In [1]:
# !pip uninstall emot

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import re
from collections import Counter
import matplotlib.pyplot as plt
from collections import Counter
import re
import emoji
# import stemmer
import string
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [3]:
def seperate_date_time(x):
    date = pd.to_datetime(x.split(', ')[0], format="%m/%d/%y")
    time = x.split(', ')[1].split(' - ')[0]
    return date, time

In [4]:
def createDf(file_data, dt):
    patterns = {
        '12 Hour':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s[APap][mM]\s-\s',
        '24 Hour':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s',
    }
    
    data = ' '.join(file_data.read().split('\n'))
    user_message = re.split(patterns[dt], data)[1:]
    date_time = re.findall(patterns[dt], data)
    
    df = pd.DataFrame({'UserMessage': user_message, 'DateTime': date_time})
    df[['Date','Time']] = df['DateTime'].apply(lambda x: seperate_date_time(x)).to_list()
    users = []
    messages = []
    for message in df['UserMessage']:
        entry = re.split('([\w\W]+?):\s', message)
        if entry[1:]:
            users.append(entry[1])
            messages.append(entry[2])
        else:
            users.append('group_notification')
            messages.append(entry[0])

    df['User'] = users
    df['Message'] = messages
    df.drop(columns=['UserMessage', 'DateTime'], inplace=True)
    return df

In [5]:
f = open('WhatsApp Chat with Say GT (original) 12(a).txt', 'r', encoding='utf-8')
df = createDf(f, '12 Hour')

In [6]:
# f = open('WhatsApp Chat with Say GT (original) 24 hour format.txt', 'r', encoding='utf-8')
# df = pd.DataFrame(createDf(f, '24 Hour'))

In [7]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month_name()
df['MonthNum'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayName'] = df['Date'].dt.day_name()
df['Hour'] = df['Time'].apply(lambda x: x.split(':')[0])
df['Minute'] = df['Time'].apply(lambda x: (x.split(':')[1]).split(' ')[0])

#Uncomment while working with 12 hour file format
# df['Meridian'] = df['Time'].apply(lambda x: (x.split(':')[1]).split(' ')[1])  

df.head()

,Date,Time,User,Message,Year,Month,MonthNum,Day,DayName,Hour,Minute
0,2022-05-09,3:01 PM,group_notification,Messages and calls are end-to-end encrypted. N...,2022,May,5,9,Monday,3,01
1,2022-05-26,7:21 PM,Clg Divyansh,Robotics ke notes,2022,May,5,26,Thursday,7,21
2,2022-05-26,7:44 PM,Clg Kushagra Ahire,MST me jo numerical aaye the uske solution bhe...,2022,May,5,26,Thursday,7,44
3,2022-05-26,7:55 PM,Clg Abbas Amjhera,<Media omitted>,2022,May,5,26,Thursday,7,55
4,2022-05-26,8:05 PM,+91 6239 413 783,Bhai notes bhej do yrr koi 🙏🏻,2022,May,5,26,Thursday,8,05


# Chatting From

In [8]:
def chat_from(selected_user, df):
    if selected_user != 'Overall':
        df = df[df['User'] == selected_user]
    unique_years = df['Year'].unique()
    start_year = unique_years[0]
    msg_count = df.groupby(['Date']).count()['Message']
    avg_msg = round(msg_count.mean(),2)
    return start_year, avg_msg

In [9]:
chat_from('Overall', df)

(2022, 22.42)

# Most Talkative

In [10]:
def most_talkative(df):
    df = df[df['User'] != 'group_notification']
    user = df['User'].value_counts()
    username = user.index[0]
    avg_msg = round(len(df)/user[username], 2)
    return username, avg_msg

In [11]:
most_talkative(df)

('Clg Ashvin birla', 6.61)

# Hourly Timeline

In [12]:
def hourly_timeline(selected_user, df, format):
    if selected_user != 'Overall':
        df = df[df['User'] == selected_user]
    new_df = df[df['User'] != 'group_notification']

    new_df['Message'] = [1] * new_df.shape[0]
    if format == '12 Hour':
        new_df['Hour'] = new_df['Hour'].astype(str) + ' ' + new_df['Meridian'].astype(str)
        new_df = new_df.groupby('Hour').sum().reset_index()
    else:
        new_df = new_df.groupby('Hour').sum().reset_index()
    return new_df

In [13]:
hourly_timeline('Overall', df, '24 Hour')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3248\3606417362.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Message'] = [1] * new_df.shape[0]


,Hour,Message,Year,MonthNum,Day
0,1,210,424620,1338,1270
1,10,109,220398,743,1400
2,11,204,412488,1419,3233
3,12,175,353850,1155,1945
4,2,71,143562,478,1004
5,3,120,242640,746,1838
6,4,71,143562,443,1085
7,5,98,198156,701,904
8,6,39,78858,266,404
9,7,30,60660,196,403


# Daily Timeline

In [14]:
def daily_timeline(selected_user, df):
    if selected_user != 'Overall':
        df = df[df['User'] == selected_user]
    new_df = df[df['User'] != 'group_notification']

    new_df['Message'] = [1] * new_df.shape[0]
    new_df = new_df.groupby('Date').sum().reset_index()
    return new_df

In [15]:
daily_timeline('Overall', df)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3248\2274322411.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Message'] = [1] * new_df.shape[0]


,Date,Message,Year,MonthNum,Day
0,2022-05-26,16,32352,80,416
1,2022-05-27,14,28308,70,378
2,2022-05-28,5,10110,25,140
3,2022-05-29,8,16176,40,232
4,2022-05-30,37,74814,185,1110
...,...,...,...,...,...
56,2022-08-12,23,46506,184,276
57,2022-08-15,8,16176,64,120
58,2022-08-16,5,10110,40,80
59,2022-08-17,54,109188,432,918


# Weekly Timeline

In [16]:
def weekly_timeline(selected_user, df):
    if selected_user != 'Overall':
        df = df[df['User'] == selected_user]
    new_df = df[df['User'] != 'group_notification']

    new_df['Message'] = [1] * new_df.shape[0]
    days = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    new_df = new_df.groupby('DayName').sum().reindex(days)
    new_df['Days'] = ['1 - Monday', '2 - Tuesday', '3 - Wednesday', '4 - Thursday', '5 - Friday', '6 - Saturday', '8 - Sunday']
    return new_df

In [17]:
weekly_timeline('Overall', df)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3248\2702942587.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Message'] = [1] * new_df.shape[0]


,Message,Year,MonthNum,Day,Days
DayName,,,,,
Monday,178,359916,1093,4117,1 - Monday
Tuesday,96,194112,720,946,2 - Tuesday
Wednesday,329,665238,2223,2488,3 - Wednesday
Thursday,199,402378,1377,2520,4 - Thursday
Friday,356,719832,2313,3779,5 - Friday
Saturday,123,248706,844,1944,6 - Saturday
Sunday,161,325542,1182,1849,8 - Sunday


# Monthly Timeline

In [18]:
def monthly_timeline(selected_user, df):
    if selected_user != 'Overall':
        df = df[df['User'] == selected_user]
    new_df = df[df['User'] != 'group_notification']
    new_df  = df.groupby(['Year','Month','MonthNum']).count()
    new_df.reset_index(inplace=True)
    time = []
    for i in range(new_df.shape[0]):
        time.append(str(new_df['MonthNum'][i]) + " - " + str(new_df['Year'][i]) + " - " + new_df['Month'][i])
    new_df['Time'] = time
    return new_df

In [19]:
monthly_timeline('Overall', df)

,Year,Month,MonthNum,Date,Time,User,Message,Day,DayName,Hour,Minute
0,2022,August,8,425,8 - 2022 - August,425,425,425,425,425,425
1,2022,July,7,343,7 - 2022 - July,343,343,343,343,343,343
2,2022,June,6,603,6 - 2022 - June,603,603,603,603,603,603
3,2022,May,5,86,5 - 2022 - May,86,86,86,86,86,86


# Most Common Word

In [20]:
def most_common_words(selected_user, df):
    if selected_user != 'Overall':
        df = df[df['User'] == selected_user]
    new_df = df[df['User'] != 'group_notification']
    new_df = new_df[~new_df['Message'].str.contains('<Media omitted>')]
    f = open('stop_hinglish.txt')
    stop_words = f.read()

    words = []

    for message in new_df['Message']:
        for word in message.lower().split():
            if word not in stop_words:
                words.append(word)

    most_common_df = pd.DataFrame(Counter(words).most_common(20), columns=['Message', 'Count'])
    most_common_df.sort_values('Count')
    return most_common_df

In [21]:
most_common_words('Overall', df)

,Message,Count
0,message,69
1,deleted,67
2,tcs,42
3,form,25
4,jain,23
5,mail,20
6,digital,19
7,drive,18
8,option,17
9,apply,17


# Donut Chart

In [22]:
def author_chart(df):
    df = df[df['User'] != 'group_notification']
    user = df['User'].value_counts()
    new_df = pd.DataFrame ({ 'User': user.index, 'Message': user})
    new_df['Percentage'] = new_df['Message'].apply(lambda x: round(x/len(df)*100, 2))
    new_df.reset_index(drop=True,inplace=True)
    return new_df

In [23]:
author_df = author_chart(df)
author_df

,User,Message,Percentage
0,Clg Ashvin birla,218,15.12
1,Clg Abhishek Jain,164,11.37
2,Clg Kushagra Ahire,130,9.02
3,Clg Amit Pandey,125,8.67
4,Clg Abbas Amjhera,118,8.18
5,Clg Divyansh,60,4.16
6,Clg Charchil,60,4.16
7,Clg Anmol Joshi,56,3.88
8,+91 96309 40258,48,3.33
9,+91 72409 15291,47,3.26


# User left

In [24]:
def removed_left(df):
    df = df[df['User']=='group_notification']
    df = df[df['Message'].str.contains('changed|deleted|encrypted|created') == False]
    removedLeft = df[df['Message'].str.contains('removed|left') ==  True]
    addedJoined = df[df['Message'].str.contains('added|joined') == True]
    removedLeft['Status'] = 0
    addedJoined['Status'] = 1
    removedLeft['User'] = removedLeft.Message.apply(lambda x: seperate(x, 'removed'))
    addedJoined['User'] = addedJoined.Message.apply(lambda x: seperate(x, 'added'))
    new_df = pd.concat([removedLeft, addedJoined])
    new_df.drop(columns=['Time', 'Year', 'Month', 'MonthNum', 'Day', 'DayName', 'Hour', 'Minute'], inplace=True, axis=1)
    return new_df

def seperate(x, seperator):
    if seperator == 'removed':
        if x.find(seperator) != -1:
            return x.split(seperator)[1]
        else:
            return x.split('left')[0]
    else:
        if x.find(seperator) != -1:
            return x.split(seperator)[1]
        else:
            return x.split('joined')[0]

# URL

In [25]:
def get_urls(df):
    urls_list = []
    url = []
    for i in df['Message']:
        o = re.findall('(https://.*)|(http://.*)',i)
        if len(o) != 0:
            urls_list.append(o[0][0].split(' ')[0])
    for i in urls_list:
        o = re.search('.*(.com)',i)
        if o != None:
            url.append(o.group())
    count_dict=Counter(url)
    count=sorted(count_dict, key=count_dict.get, reverse=True)
    return count_dict,count

In [26]:
links_dictionary,links = get_urls(df)
for link in links:
    print(link,links_dictionary[link])

https://teams.microsoft.com 4
https://docs.google.com 3
https://chat.whatsapp.com 2
https://www.tcs.com 1
https://meet.google.com 1
https://youtube.com 1


# emoji

In [27]:
def emoji_helper(selected_user, df):
    if selected_user != 'Overall':
        df = df[df['user'] == selected_user]
    emojis = []
    description = []
    for message in df['Message']:
        all_emoji = emoji.distinct_emoji_list(message)
        emojis.extend([emoji.emojize(is_emoji) for is_emoji in all_emoji])
    emoji_df = pd.DataFrame(Counter(emojis).most_common(len(Counter(emojis))))
    for i in emoji_df[0]:
        description.append(emoji.demojize(i))
    emoji_df['description'] = description
    return emoji_df[:5],emoji_df

In [28]:
edf=emoji_helper('Overall' ,df)
edf

(   0   1                      description
 0  😂  40         :face_with_tears_of_joy:
 1  🤣  23  :rolling_on_the_floor_laughing:
 2  😅  20       :grinning_face_with_sweat:
 3  🙂  20          :slightly_smiling_face:
 4  🥳  11                  :partying_face:,
        0   1                         description
 0      😂  40            :face_with_tears_of_joy:
 1      🤣  23     :rolling_on_the_floor_laughing:
 2      😅  20          :grinning_face_with_sweat:
 3      🙂  20             :slightly_smiling_face:
 4      🥳  11                     :partying_face:
 5      🙄   7            :face_with_rolling_eyes:
 6      💐   7                           :bouquet:
 7      😁   7    :beaming_face_with_smiling_eyes:
 8      🥲   6            :smiling_face_with_tear:
 9      🙏   5                      :folded_hands:
 10     👍   5                         :thumbs_up:
 11     🎉   4                      :party_popper:
 12    ☹️   4                     :frowning_face:
 13  💁‍♂️   4                  :man_tippi

# most emoji shared by user

In [29]:
def get_emoji(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
                      "]+", re.UNICODE)
    obj = re.findall(emoj,data)
    return obj

In [30]:
test_df = df[df['User']!='group_notification']
test_df = df[['User','Message']]
test_df = test_df[test_df['User']!='group_notification']
test_df['Message'] = test_df.groupby(['User'])['Message'].transform(lambda x : ' '.join(x))
test_df = test_df.drop_duplicates()
emoji_user = {}
for i in range(test_df.shape[0]):
    emoji_user[test_df.iloc[i]['User']] = len(get_emoji(test_df.iloc[i]['Message']))

In [31]:
name = max(emoji_user, key=emoji_user.get)
# emoji_percent = emoji_user[name]/
name

'Clg Ashvin birla'

In [32]:
d=test_df[test_df['User']==name]

# long winded

In [33]:
def long_winded(df):
    msg_len = []
    for i in df['Message']:
        msg_len.append(len(i))
    df['Message_len'] = msg_len
    new_df = df.sort_values(by=['Message_len'],ascending=False)
    # apply mean in message_len sum /len
    name = new_df.iloc[0]['User']
    user_df = df[df['User']==name]
    length = new_df.iloc[0]['Message_len']
    user_df = df[df['User']==name]
    mean = user_df['Message_len'].mean()
    mean = round((mean/100),2)
    df.drop(['Message_len'],axis=1,inplace=True)
    return name,length,mean

In [34]:
long_winded(df)

('Clg Akshat Khujneri', 695, 1.05)

# influencer

In [35]:
def influencer(df):
    df=df[df['User']!='group_notification']
    new_df = df.groupby(['User'])
    inf_dict = {}
    for name, group in new_df:
        count = 0
        for i in group['Message']:
            if '<Media omitted>' in i:
                count += 1
        inf_dict[name]=count
    name = max(inf_dict, key=inf_dict.get)
    name_df = df[df['User']==name]
    percent = (inf_dict[name]/name_df.shape[0])*100
    return name,round(percent,2)
# groupby user and media omitted

In [36]:
print(influencer(df))

('Clg Abbas Amjhera', 13.56)


# professor

In [37]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [38]:
def remove_punctuations(string):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for ele in string:
        if ele in punc:
            string = string.replace(ele, "")
    return string

In [39]:
def professor(df):
    professor_dict = {}
    words_list = []
    lancaster=LancasterStemmer()
    test_df = df[['User','Message']]
    test_df = test_df[test_df['User']!='group_notification']
    test_df['Message'] = test_df.groupby(['User'])['Message'].transform(lambda x : ' '.join(x))
    test_df = test_df.drop_duplicates()
    for i in range(test_df.shape[0]):
        stemmed_words = []
        s = test_df.iloc[i]['Message']
        text = remove_emojis(s)
        text = remove_punctuations(text)
        words_list = text.split()
        words_list = list(set(words_list))
        for w in words_list:
            stemmed_words.append(lancaster.stem(w))
        professor_dict[test_df.iloc[i]['User']]=len(stemmed_words)
    name = max(professor_dict, key=professor_dict.get)
    return name,professor_dict[name]

In [40]:
professor(df)

NameError: name 'LancasterStemmer' is not defined

# sentimental analysis 

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [42]:
from sklearn.metrics import accuracy_score

In [43]:
import stemmer

In [44]:
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [45]:
def text_transformation(words_list):
    corpus = []
    for item in words_list:
        new_item = re.sub('[^a-zA-Z]',' ',str(item))
        new_item = new_item.lower()
        if 'https' in new_item or 'http' in new_item:
            pass
        else:
            new_item = new_item.split()
            for i in new_item:
                corpus.append(i)
    return list(set(corpus))

In [101]:
# train test split
from sklearn.model_selection import train_test_split
words_df = pd.read_excel('New_Words.xlsx')
words_df.dropna(inplace=True)
X = words_df['Word']
y = words_df['Label']

In [102]:
words_df.isnull().sum()

Word     0
Label    0
dtype: int64

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [110]:
# train
cv = CountVectorizer(ngram_range=(1,2))
traindata = cv.fit_transform(list(X_train))
X = traindata
y = y_train
rfc = RandomForestClassifier()
rfc.fit(X,y)

RandomForestClassifier()

In [105]:
X.shape,y.shape

((5569, 5559), (5569,))

In [111]:
testdata = cv.transform(X_test)
predictions = rfc.predict(testdata)

In [107]:
predictions

array([0., 0., 0., ..., 0., 0., 0.])

In [112]:
acc_score = accuracy_score(y_test,predictions)

In [113]:
print(acc_score)

0.16511127063890882


In [ ]:
td = cv.transform(message_words_list)
p = rfc.predict(td)

In [61]:
rfc.predict(cv.transform(['bekar','bura','enjoy'])).reshape(-1)

array([1., 1., 1.])

In [42]:
# converting the messages to words_list
test_df = df[df['User']!='group_notification']
test_df = test_df[test_df['Message']!='<Media omitted> ']
test_df = test_df[test_df['Message']!='This message was deleted'] 

In [43]:
message_words_list = text_transformation(test_df['Message'])

In [ ]:
string_words = []
for i in X:
    string_words.append(str(i))

In [44]:
myStemmer = stemmer.Stemmer()
stemmed_list = myStemmer.stemListOfWords(string_words)

In [46]:
type(stemmed_list)

list

In [ ]:
def text_transformation(words_list):
    corpus = []
    myStemmer = stemmer.Stemmer()
    for item in words_list:
        new_item = re.sub('[^a-zA-Z]',' ',str(item))
        new_item = new_item.lower()
        if ('https://' or 'http://') in new_item:
            pass
        else:
            new_item = new_item.split()
            for i in new_item:
                i = myStemmer.stemWord(str(i))
                if len(i)>1:
                    corpus.append(i)
    return list(set(corpus))